# Testing

# What is (software) testing?

* Excuting a program with the intent of finding software bugs.
* Collects information about the quality of a product or service.
* Demonstrates that using a software in specific ways gives the expected results.
* etc etc etc

# The `unittest` module 

* xUnit similar to jUnit
* each test case is a class inherting from `unittest.TestCase`
* test runner executes all `test_xxx` methods
* test methods can `assertXXX()` conditions
* test results can be printed, stored as XML etc.

## A simple function to test

In [1]:
def divided(dividend, divisor):
    return dividend / divisor

## The test case

In [2]:
# Simple test case to test a trivial ``divided`` function.
import unittest


class DivideTest(unittest.TestCase):
    def test_can_divide_positive_numbers(self):
        self.assertEqual(3, divided(15, 5))
        self.assertAlmostEqual(2.5, divided(5, 2))

    def test_can_divide_negative_numbers(self):
        self.assertEqual(-3, divided(15, -5))

    def test_can_divide_zero(self):
        self.assertEqual(0, divided(0, 1))
        self.assertEqual(0, divided(0, -1))
        self.assertEqual(0, divided(0, 123.45))

## Testing for expected errors

In [3]:
class DivideTest(unittest.TestCase):
    # ...
    def test_fails_on_zero_division(self):
        self.assertRaises(ZeroDivisionError, divided, 1, 0)

Note that `divided()` is not called here, only the function and its parameters is passed.

## Common `assertXXX()` methods

* `asserEqual(a, b)` - check that `a == b`
* `assertAlmostEqual(a, b)` - check that two floating point numbers have almost the same value
* `assertRegex(text, regex)` - check that a `text` matches a regular expression
* `assertIn(needle, haystack)` - check that `needle` is in `haystack`
* `assertIsNone(value)` - check that `value` is `None`
* `assertTrue(expression)`, `assertFalse(expression)` - check the result of any expression

## Common `assertXXX()` methods (continued)

All methods have an optional argument `message` to specify your own message in case of error. Typically the standard messages are adequate though with the exception of `assertTrue()` and `assertFalse()`.

Most methods also have a `Not` equivalent, e.g. `assertEqual()` and `assertNotEqual()`.

For a full list, visit https://docs.python.org/3/library/unittest.html.

## Test runners

* Test runners look for classes inheriting from `unittest.TestCase` (or children of it) and run all `test_xxx(self)` methods in it.
* Some test runners also can recursively discover test cases across multiple folders.
* If a `test_xxx()` fails, the test runner continues with test next test method.
* The test runner collects all results.

## Let's break things

Change the division operator (`/`) to integer division (`//`) to break one of our test cases, so the test results get more interesting:

In [4]:
def divided(dividend, divisor):
    return dividend // divisor

## Command line test runner

The `unittest` module already provides a built in test runner that can be called from the command line by simply adding the following lines to the test case:

```python
if __name__ == '__main__':
    unittest.main()
```

## Command line test runner: example output

```
.F..
===================================================
FAIL: test_can_divide_positive_numbers (__main__.DivideTest)
---------------------------------------------------
Traceback (most recent call last):
  File "/home/roskakori/workspace/talks/python_for_testers/examples/test_divide.py", line 12, in test_can_divide_positive_numbers
    self.assertAlmostEqual(2.5, divided(5, 2))
AssertionError: 2.5 != 2 within 7 places

---------------------------------------------------
Ran 4 tests in 0.001s

FAILED (failures=1)
```

## IDE test runners

![Screenshot: PyCharm test runner](examples/pycharm_test_runner.png)

## Failures and errors

* Failures
  * indicate that a test condition using `assertXXX()` was not met.
  * are often fixed by modifying the code of the application under test.
* Errors
  * indicate that the test failed because an `Exception` was raised, e.g. because a test file was missing or a network connection could not be established.
  * are often fixed by modifying the test environment.
  * might sometimes hint at incomplete error handling.

## Intermission: running a single test case

This is a little helper function to run all `test_*()` methods in a `TestCase`. This is only required to run our example tests directly in the notebook. For all practical purpose, use the existing test runners.

In [5]:
def run_test_class(test_class):
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(test_class)
    unittest.TextTestRunner().run(suite)

## Example failure and error

In [6]:
class FailureAndErrorTest(unittest.TestCase):
    def test_error(self):
        with open('no_such_file.tmp', 'r', encoding='utf-8'):
            assertEqual(1, 0)

    def test_failure(self):
        self.assertEqual(1, 0)        

run_test_class(FailureAndErrorTest)

EF
ERROR: test_error (__main__.FailureAndErrorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-7e2ee2724fb4>", line 3, in test_error
    with open('no_such_file.tmp', 'r', encoding='utf-8'):
FileNotFoundError: [Errno 2] No such file or directory: 'no_such_file.tmp'

FAIL: test_failure (__main__.FailureAndErrorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-7e2ee2724fb4>", line 7, in test_failure
    self.assertEqual(1, 0)
AssertionError: 1 != 0

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (failures=1, errors=1)


## Test fixtures

* A test fixture "is all the things that must be in place in order to run a test" (Wikipedia)
* Less abstract: all the resources used by every `test_*()` method in a `TestCase`.
* Examples:
  * instance variables pointing to often used file system paths
  * database or network connections
* Use `setUp()` to set or allocate the resources and `tearDown()` to release them.

## Example test fixture

In [7]:
import os.path
import unittest

class TextFileTest(unittest.TestCase):
    def setUp(self):
        text_path = os.path.join('examples', 'der_rote_komet.txt')
        self._text_file = open(text_path, 'r', encoding='utf-8')

    def tearDown(self):
        self._text_file.close()

    def test_has_lines(self):
        line_count = len(list(self._text_file))
        self.assertGreater(line_count, 0)
        
    def test_has_line_with_umlaut_u(self):
        self.assertTrue(any('ü' in line for line in self._text_file))

run_test_class(TextFileTest)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


## Extending `TestCase`

It can be helpful to extend `TestCase` with additional common functions used by tests for your software, in particular additional `assertXXX()`. Example:

In [8]:
class FileTestCase(unittest.TestCase):
    def assertFileExists(self, path):
        self.assertTrue(
            os.path.exists(path),
            'file must exist: "{0}"'.format(path))

    def assertFileHasData(self, path):
        self.assertGreater(
            os.path.getsize(path), 0,
            'file must contain data: "{0}"'.format(path))

# The `pytest` module

* Available from http://pytest.org
* Uses `assert` statement and meta programming for detailed error messages about failed tests
* Supports multiple paradigms, can be used without object orientation
* Can easily find and run tests in complex folder structures

## Testing `divided()` with `pytest`

In [9]:
# %load examples/test_divided_using_pytest.py
# Test a division function using pytest.
def divided(dividend, divisor):
    return dividend / divisor


def assert_almost_equal(a, b, places=7):
    assert round(abs(a - b), places) == 0


def test_can_divide_positive_numbers():
    assert 3 == divided(15, 5)
    assert_almost_equal(2.5, divided(5, 2))


def test_can_divide_negative_numbers():
    assert -3 == divided(15, -5)


def test_can_divide_zero():
    assert 0 == divided(0, 1)
    assert 0 == divided(0, -1)
    assert 0 == divided(0, 123.45)

## Running `py.test`

Command line call:
```
py.test examples/test_divided_using_pytest.py
```
Result:
```
============ test session starts =============
platform linux -- Python 3.5.1, pytest-2.8.1, ...
rootdir: /home/.../examples, inifile: 
collected 3 items 

examples/test_divided_using_pytest.py ...
========== 3 passed in 0.02 seconds ==========
```

## `pytest` and test failures

```
______ test_can_divide_positive_numbers ______

    def test_can_divide_positive_numbers():
        assert 3 == divided(15, 5)
>       assert_almost_equal(2.5, divided(5, 2))

examples/test_divided_using_pytest.py:12: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

a = 2.5, b = 2, places = 7

    def assert_almost_equal(a, b, places=7):
>       assert round(abs(a - b), places) == 0
E       assert 0.5 == 0
E        +  where 0.5 = round(0.5, 7)
E        +    where 0.5 = abs((2.5 - 2))
```


# Summary

* `unittest` is a traditional xUnit test framework
  * object oriented
  * uses `assertXXX()`
* `pytest` is a popular Python specific test framework
  * supports multiple paradigms
  * uses plain `assert`
  * uses meta programming for very specific error messages
  * powerful test discovery